# IMDb Sentiment Analysis with Bidirectional LSTM

This notebook builds a sentiment classifier for IMDb movie reviews using TensorFlow/Keras. We walk through loading the dataset, preparing padded sequences, training a bidirectional LSTM, and evaluating performance.

**Pipeline overview**

1. Setup and imports  
2. Load and inspect the IMDb dataset  
3. Preprocess sequences with padding and exploratory checks  
4. Build the Bidirectional LSTM Model
5. Evaluate accuracy and error patterns, then outline next steps

Dataset: TensorFlow/Keras IMDb reviews (25k train / 25k test) with integer-encoded tokens sorted by frequency.


## 1. Setup and Imports

Configure core libraries, reproducibility seeds, and training hyperparameters used throughout the workflow.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Check working directory
print(f"Working directory: {os.getcwd()}")

## 2. Load and inspect the IMDb dataset


In [ ]:
# 1️⃣ Load each split dataset from CSV
df_o = pd.read_csv("imdb_adaptation.csv")

print("\n🔹 Training set structure:")
df_o.info()          # Overview of columns, dtypes, and non-null counts
print("\n🔹 First rows of training data:")
print(df_o.head())   # Display first few rows to confirm format

## 3. Preprocess sequences with padding and exploratory checks
This step uses padding and systematic data splitting to prepare the dataset for neural network training.
It ensures that all sequences share the same dimensionality, that the data distribution remains consistent, and that the model can be properly validated and tested on unseen data.

### 3.1 Padding

In [ ]:
# Convert string representations of lists back to actual lists
from ast import literal_eval
df_o["sequence"] = df_o["sequence"].apply(literal_eval)

# Apply padding separately
MAX_LEN = 200
X_padded = pad_sequences(df_o["sequence"], maxlen=MAX_LEN)

# Extract labels
y = df_o["label"].values

# Display some information about the processed data
print("First converted sequence:", df_o["sequence"].iloc[0])
print("Shape after padding:", X_padded.shape)


### 3.2 Split Train/Val/Test -> 80/10/10

In [ ]:
# 1️⃣ First split → 80% train / 20% temp (stratified)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_padded, y,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=y
)

# 2️⃣ Split the 20% into 10% val / 10% test (stratified)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    shuffle=True,
    stratify=y_temp
)

print("✅ Stratified data split done:")
for name, arr in zip(["Train", "Validation", "Test"], [y_train, y_val, y_test]):
    unique, counts = np.unique(arr, return_counts=True)
    ratio = counts / counts.sum()
    print(f"{name} class ratio: {dict(zip(unique, np.round(ratio, 3)))}")

#  DISPLAY FINAL SHAPES
print("\n✅ Final dataset shapes:")
print(f"Train:       {X_train.shape}")
print(f"Validation:  {X_val.shape}")
print(f"Test:        {X_test.shape}")

## 4. Build the Bidirectional LSTM Model

Stack an embedding layer with a bidirectional LSTM to capture contextual information from both directions, then pool and classify with dense layers.


In [ ]:
# ============================================
# BUILD MODEL
model = Sequential([
    tf.keras.layers.Input(shape=(MAX_LEN,)),
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, mask_zero=True),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units=32,
            dropout=0.4,
            recurrent_dropout=0.4
        )
    ),

    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model with optimizer, loss, and evaluation metric
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# ============================================
# TRAIN THE MODEL
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5, verbose=1
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)


In [ ]:
# PLOT TRAINING CURVES
plt.plot(history.history['loss']) # Plotting training loss
plt.plot(history.history['val_loss']) # Plotting validation loss
plt.title("Training and Validation Loss Over Epochs") # Adding title
plt.xlabel("Epochs") # Adding x-axis label
plt.ylabel("Loss") # Adding y-axis label
plt.legend(["Training Loss", "Validation Loss"])
plt.show() # Displaying the plot

## 5. Evaluate accuracy and error patterns

After training, the model’s performance is assessed on the test set to verify its ability to generalize to unseen data.
Besides the loss and accuracy metrics automatically tracked during training, a more detailed analysis is carried out using additional classification metrics such as Precision, Recall and F1 Score.

These metrics provide a clearer understanding of how well the model distinguishes between positive and negative reviews, complementing the overall accuracy measure.

In [ ]:
# Evaluate on test set using Keras built-in evaluation
loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy (Keras): {test_accuracy:.4f}")

# Manual evaluation using sklearn metrics
y_pred = (model.predict(X_test) > 0.5).astype("int32")

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy (Sklearn): {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)


## Next Steps

- Explore alternative model architectures (stacked LSTMs, GRUs, or convolutional front-ends)
- Increase `NUM_WORDS` or `MAX_LEN` and observe the trade-off between vocabulary coverage and training cost.
- Add regularisation such as dropout, or incorporate pre-trained word embeddings (e.g., GloVe) for richer representations.
